### Author: Bogdan Bintu and Benjamin Albert
     Date: 10/6/2022
    This is inteded to create genome-scale probes with ~250kb resolution across the human genome

In [1]:
!cd LibraryDesign3/C_Tools && python setup.py build_ext --inplace

running build_ext
copying build/lib.linux-x86_64-3.9/seqint.cpython-39-x86_64-linux-gnu.so -> 


In [1]:
import sys,os
libdesignpath = "LibraryDesign3"
if libdesignpath not in sys.path:
    sys.path.append(libdesignpath)
import LibraryDesigner as ld # adds C_Tools to path
import LibraryTools as lt
from seqint import seq2Int, seq2Int_rc
# some other packages may required
import Bio #use pip install biopython

ModuleNotFoundError: No module named 'LibraryDesigner'

In [ ]:
#### Downloaded T2T genome from: https://www.ncbi.nlm.nih.gov/assembly/GCA_009914755.4
#### GCF_009914755.1_T2T-CHM13v2.0_genomic.fna

In [ ]:
names,seqs = lt.fastaread("data/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna")

### Construct count tables -- only need to do it once

In [ ]:
fl = "data/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna"
names,seqs = lt.fastaread(fl)
ct = ld.countTable(word=17,save_file=fl.replace(".fa","_17w.bin"),
                   sparse=False)

from tqdm import tqdm
for sq in tqdm(seqs):
    ct.consume(sq,verbose=False)
ct.complete(verbose=True)
ct.save()

In [ ]:
import numpy as np
#arr = np.load(r'C:\Users\Bogdan\Dropbox\HumanGenomeTranscriptome\human_genome_T2T_17w.bin',allow_pickle=True)
arr = np.fromfile("data/human_genome_T2T_17w.bin",dtype=np.uint16)

seq = b"ccctaaccctaacccta"
arr[ld.seq2Int(seq)] #number of occurences in the genome of ccctaaccctaacccta

In [ ]:
# TODO

In [ ]:
fl = "data/human__genesUnspliced.fasta"
names,seqs = lt.fastaread(fl)
ct = ld.countTable(word=17,save_file=fl.replace(".fasta","_17w.bin"),
                   sparse=False)

from tqdm import tqdm
for sq in tqdm(seqs):
    ct.consume(sq,verbose=False)
ct.complete(verbose=True)
ct.save()

### sample genome at 250kb and save 50kb centered

TODO

Save these as fata files index by chr and location: chr5_locus_0025_50kb.fasta

### Run on single sequence the probe designing

In [ ]:
names[4]
save_folder = "probes"
ich=4
index = 25

name = "chr5_locus"+str(index).zfill(5)+"_50kb"
seq = seqs[ich][index*250000:][:50000]
fl_name = save_folder+os.sep+name+".fasta"
lt.fastawrite(fl_name,[name],[seq])

In [ ]:
print(fl_name)

In [ ]:
in_file = "data/chr5_locus00025_50kb.fasta"

save_file = in_file.replace('.fasta','.pbr')

transcriptome_fl = "data/human__genesUnspliced_17w.bin"
genome_fl = "data/human_genome_T2T_17w.bin"
rep_fl = "data/repetitive.fasta"
top_transcriptome_fl = "data/top_tr500_brain.fasta"
local_genome_fl = in_file


pb_designer = ld.pb_reports_class(
    sequence_dic={'file':in_file,'use_revc':True,'use_kmer':True}, #use_revc - if true  consides rc as probe
    
    map_dic={'transcriptome':{'file':transcriptome_fl,'use_revc':False,'use_kmer':True},
             'top_transcriptome':{'file':top_transcriptome_fl,'use_revc':False,'use_kmer':True},
          'genome':{'file':genome_fl,'use_revc':True,'use_kmer':True},
          'repetitive':{'file':rep_fl,'use_revc':True,'use_kmer':True},
          'local_genome':{'file':in_file,'use_revc':True,'use_kmer':True}},

    save_file=save_file,
    
    params_dic={'word_size':17,'pb_len':40,'buffer_len':0,'max_count':2**16-1,'check_on_go':False,'auto':False},
    
    dic_check={('genome','local_genome'):75,
                'transcriptome':20,
                'top_transcriptome':5,
                'repetitive':2,
               'gc':[0.25,0.75],'tm':70})

pb_designer.computeOTmaps()
pb_designer.compute_pb_report(verbose=True)
pb_designer.perform_check_end()
pb_designer.plots()
pb_designer.save_csv(name=os.path.basename(in_file).split('.')[0])

In [ ]:
#### Define in_files

In [ ]:
from tqdm.notebook import tqdm
for in_file in tqdm(in_files):
    #print(in_file)
    #modify input file
    pb_designer.sequence_dic['file'] = in_file
    #modify save file
    pb_designer.save_file = in_file.replace('.fasta','.pbr')
    if not os.path.exists(pb_designer.save_file):
        pb_designer.load_sequence_file_and_paramaters()
        #modify maps
        key='local_genome'
        pb_designer.map_dic[key]['file'] = in_file
        pb_designer.files_to_OTmap("map_"+key,pb_designer.map_dic[key])
        #compute
        pb_designer.compute_pb_report()
        pb_designer.perform_check_end()
        pb_designer.plots()
        pb_designer.save_csv(name=os.path.basename(in_file).split('.')[0])

In [ ]:
np.sum([int(len(seq)/250000) for seq in seqs])